# Group 27

# Text Classification

# Import packages

In [1]:
import numpy as np
#To load the datasets
import os
#to remove puntuation 
import string
#To preprocess the datasets
from nltk.corpus import stopwords #(tokonization)
from sklearn.feature_extraction.text import CountVectorizer
import re, string
#packages for model selection (SVM)
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import metrics
from scipy import sparse
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
#GridSearh
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
#Time
import time

# Load the datasets

In [2]:
def Open_Files (path):
    content = []
    file_names = os.listdir(path)
    for file in file_names:
        t = path + file
        content.append(open(t, encoding ='utf-8').read())  #English Text
    return content

In [3]:
training_pos = Open_Files('./train/pos/')
training_neg = Open_Files('./train/neg/')
test_pos = Open_Files('./test/pos/')
test_neg = Open_Files ('./test/neg/')


# Preprocessing for Textual Data 


The function reads the file, removes punctuation marks (by translator), and puts everything in lower case. 
Coding BOW algorithm, the input will be multiple scentences and the output will be the vectors.
- Step (1), tokenization: 
  - Discard some characters like punctuation marks.
  - Remove Stopwords (do not give significant values to our model) by sorting a list of words. For example,"is, the, a" 
  - Robust implementation of stopwords (nltk library). It has a set of predefined words per language.
- step (2), apply tokenization to all scentences. 
- step (3), build vocabulary and generate vectors.
  


In [4]:
def preprocessing(file):
    text_content = []
    exclude = string.punctuation
    exclude = exclude.replace("-", "")
    pattern = r"[{}]".format(exclude)

    for data in file :
        text = re.sub(r"(<br\s*/><br\s*/>)", " ", str(data))
        text = re.sub(pattern, "", str(text))
        text_content.append(text.lower())
    return text_content


Feature Extraction 
- extracts the top n frequent features with their respective ID. 
- writes output file for feature ID and count 
- writes output files for feature vectors for train, valid, test set 

This step later on (if needed)


# Data Sbliting and Vectorzation

In [5]:
#Data Spliting and Labelization 
train_valid_cv= training_pos + training_neg
train_valid_cv = preprocessing(train_valid_cv)
training_set = training_pos[:10000] + training_neg[:10000]
training_set = preprocessing(training_set)
validation_set = training_pos [10000:] + training_neg[10000:]
validation_set = preprocessing(validation_set)
test = test_pos + test_neg
test_set = preprocessing(test)
#labelization
y_train = np.append(np.ones(10000), np.zeros(10000))
y_valid = np.append(np.ones(2500), np.zeros(2500))
y_test = np.append(np.ones(12500), np.zeros(12500))
y_cv = np.append(np.ones(12500), np.zeros(12500))
#y_crossvalidation = np.append(np.ones(12500), np.zeros(12500))

In [6]:
#Tifd

from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer

tf_idf_vectorizer = TfidfVectorizer()
train_set = tf_idf_vectorizer.fit_transform(training_set)
train_valid_set = tf_idf_vectorizer.fit_transform(train_valid_cv)
valid_set = tf_idf_vectorizer.transform(validation_set)
test_set = tf_idf_vectorizer.transform(test_set)

In [10]:
print(valid_set.shape)
print(train_set.shape)
print(test_set.shape)
print(train_valid_set.shape)
print(y_cv.shape)

(5000, 95192)
(20000, 84708)
(25000, 95192)
(25000, 95192)
(25000,)


# GridSearch and Model Selection

# GridSearch: SVM with different Kernel Functions!

In [ ]:
#Define the hyper-parameters
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    t0=time.time()
    clf.fit(train_set, y_train)
    print ("training time:", round(time.time()-t0, 3), "s")
    
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    #print()
    #t1=time.time()
    #y_pred = clf.predict(tes_set)
    #print ("predict time:", round(time.time()-t1, 3), "s")
    #print(metrics.classification_report(y_tes, y_pred, digits = 5))

# GridSearch: Decision Trees

- Model Selection: Tunning Hyperparameters:

Function: 
- the objective is to find the best hyperparameter for the model (clf-Sklearn)
-Then refit the model with the paramters
-Predict train and valid 
-Calculate f1_score and accuracy
-Returns: f1_score train and f1_score valid
-Inputs: set (dataset), clf (model), params (tunning hyperparameters)



In [ ]:
def Model_Selection_train(train_inp, y_tra, valid_inp, y_vald, test_inp, y_tst, clf, params):

    train_input = train_inp
    valid_input = valid_inp
    test_input  = test_inp
    
    train_label = y_tra
    valid_label = y_vald
    test_label  = y_tst
    
    if params != None :
        '''cross-validation  
        1) First, we combine training and validation set togther
        2) Second, we feed the split into GridSearchCV (with cross validation)
        3) Third, we calculate the accuracy on training, validation, and test.
        '''
        combine_input = sparse.vstack([train_input, valid_input])
        combine_label = np.concatenate((train_label, valid_label))
        fold = [-1 for i in range(train_input.shape[0])] + [0 for i in range(valid_input.shape[0])]
        ps = PredefinedSplit(test_fold = fold)
        clf = GridSearchCV(clf, params, cv=ps, refit=True)
        clf.fit(combine_input, combine_label)
    else:
        clf.fit(train_input, train_label)
        
    best_param = None if params==None else clf.best_params_

    f1_train = f1_score(train_label, clf.predict(train_input), average = 'micro')
    f1_valid = f1_score(valid_label, clf.predict(valid_input), average = 'micro')
    f1_test = f1_score(test_label, clf.predict(test_input), average = 'micro')
    Acc_train = accuracy_score(train_label, clf.predict(train_input), normalize = True)
    Acc_valid = accuracy_score(valid_label, clf.predict(valid_input), normalize = True)
    Acc_test = accuracy_score(test_label, clf.predict(test_input), normalize = True)

    return f1_train, f1_valid, f1_test, Acc_train, Acc_valid, Acc_test, best_param

In [ ]:
#Grid search (1) for Decision Trees 
#Hyperparameters : 
# max-features = 80000
# max-depth = [i for i in range(15, 20)]
# max-leaf-nodes = [1000 * i for i in range(3, 6)] 
# min-samples-leaf = np.arange(1, 20)

param = [{'max_depth': [i for i in range(15, 20)], 'max_features': [80000], 'max_leaf_nodes': [1000 * i for i in range(3, 6)], 'min_samples_leaf': np.arange(1, 20)}]
t1=time.time()
pred = Model_Selection_train(train_set, y_train, valid_set, y_valid, test_set, y_test, DecisionTreeClassifier(), param)
print(set, "Decision Tree \n(train, valid, test) = ", pred[:6])
print("best params = {}\n".format(pred[6]))
print ("predict time:", round(time.time()-t1, 3), "s")

In [ ]:
#Grid search (2) for Decision Trees 
#Hyperparameters : 
# max-features = 80000
# max-depth = [i for i in range(15, 20)]
# max-leaf-nodes = [1000 * i for i in range(3, 6)] 
# min-samples-leaf = np.arange(1, 20)

param = [{'max_depth': [i for i in range(15, 20)], 'max_features': [1000 * i for i in range(2, 7)], 'max_leaf_nodes': [1000 * i for i in range(3, 6)], 'min_samples_leaf': np.arange(1, 20)}]
pred = Model_Selection_train(train_set, y_train, valid_set, y_valid, test_set, y_test, DecisionTreeClassifier(), param)
print(set, "Decision Tree \n(train, valid, test) = ", pred[:6])
print("best params = {}\n".format(pred[6]))

#based on a reference the minimum samples leaf (1, 20)(refrence: “An empirical study on hyperparameter tuning of decision trees)

# Testing the Final (selected) model 

# SVM 

In [12]:
############ Train+ Valid ##################
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import metrics
import time

t0=time.time()
clf = svm.SVC(kernel='rbf',C = 100, gamma = 0.001 ).fit(train_valid_set, y_cv)
print ("training time:", round(time.time()-t0, 3), "s")
t1=time.time()
y_pred = clf.predict(train_valid_set)
print ("predict time:", round(time.time()-t1, 3), "s")
print(metrics.classification_report(y_cv, y_pred, digits = 5))


training time: 451.324 s
predict time: 412.034 s
              precision    recall  f1-score   support

         0.0    0.92810   0.91280   0.92038     12500
         1.0    0.91421   0.92928   0.92169     12500

    accuracy                        0.92104     25000
   macro avg    0.92115   0.92104   0.92103     25000
weighted avg    0.92115   0.92104   0.92103     25000



In [13]:
############ Test #################
t2=time.time()
y_pred = clf.predict(test_set)
print ("predict time for test:", round(time.time()-t2, 3), "s")
print(metrics.classification_report(y_test, y_pred, digits = 5))


predict time for test: 404.358 s
              precision    recall  f1-score   support

         0.0    0.88513   0.87840   0.88175     12500
         1.0    0.87932   0.88600   0.88265     12500

    accuracy                        0.88220     25000
   macro avg    0.88222   0.88220   0.88220     25000
weighted avg    0.88222   0.88220   0.88220     25000



# Decision Trees

In [14]:
#############Train+Valid################
t0=time.time()
clfdt = DecisionTreeClassifier(max_depth= 18, max_features= 80000, max_leaf_nodes= 3000, min_samples_leaf= 8 ).fit(train_valid_set, y_cv)
print ("training time:", round(time.time()-t0, 3), "s")
t1=time.time()
y_pred = clfdt.predict(train_valid_set)
print(metrics.classification_report( y_cv, y_pred, digits = 5))
#########Test######################
t2=time.time()
y_pred = clfdt.predict(test_set)
print ("predict time for test:", round(time.time()-t2, 3), "s")
print(metrics.classification_report(y_test, y_pred, digits = 5))

training time: 16.647 s
              precision    recall  f1-score   support

         0.0    0.83802   0.77648   0.80608     12500
         1.0    0.79177   0.84992   0.81982     12500

    accuracy                        0.81320     25000
   macro avg    0.81490   0.81320   0.81295     25000
weighted avg    0.81490   0.81320   0.81295     25000

predict time for test: 0.041 s
              precision    recall  f1-score   support

         0.0    0.73908   0.68208   0.70944     12500
         1.0    0.70484   0.75920   0.73101     12500

    accuracy                        0.72064     25000
   macro avg    0.72196   0.72064   0.72022     25000
weighted avg    0.72196   0.72064   0.72022     25000



# New Models


In [16]:
######################## Model 1 #############################
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy import sparse
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=True, n_jobs=1, beta = 0.75):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs
        self.beta = beta

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        #y = y.values
        #x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        x_nb = x.multiply(self._r)
        self._clf = LinearSVC(C=self.C, dual=self.dual).fit(x_nb, y)
        w = np.append(self._clf.coef_, self._clf.intercept_)
        wbar = np.abs(w).mean(axis=0)
        self._clf.coef_ = self.beta * wbar + (1 - self.beta) * self._clf.coef_
        self._clf.intercept_ = self.beta * wbar + (1 - self.beta) * self._clf.intercept_
        return self


In [17]:
#########Train + Valid ################################
from sklearn.pipeline import Pipeline

t0=time.time()
nbSVM = NbSvmClassifier(C=1, beta =0.25).fit(train_valid_set, y_cv)
print ("training time:", round(time.time()-t0, 3), "s")

t1=time.time()
y_pred = nbSVM.predict(train_valid_set)
print ("predict time:", round(time.time()-t1, 3), "s")
print(metrics.classification_report(y_cv, y_pred, digits = 5))

################ Test #######################################
t1=time.time()
y_pred = nbSVM.predict(test_set)
print ("predict time:", round(time.time()-t1, 3), "s")
print(metrics.classification_report(y_test, y_pred, digits = 5))


training time: 1.05 s
predict time: 0.169 s
              precision    recall  f1-score   support

         0.0    0.95507   0.93864   0.94678     12500
         1.0    0.93968   0.95584   0.94769     12500

    accuracy                        0.94724     25000
   macro avg    0.94737   0.94724   0.94724     25000
weighted avg    0.94737   0.94724   0.94724     25000

predict time: 0.166 s
              precision    recall  f1-score   support

         0.0    0.89526   0.88688   0.89105     12500
         1.0    0.88793   0.89624   0.89207     12500

    accuracy                        0.89156     25000
   macro avg    0.89159   0.89156   0.89156     25000
weighted avg    0.89159   0.89156   0.89156     25000

